### Basics

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.chat_models import ChatOllama

In [68]:
llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,  # use 0.0 for deterministic outputs
    base_url="http://localhost:11434",
    api_key="ollama"
)

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

current_messages =  [
        SystemMessage(content="You are a helpful AI bot that assists a user in choosing the perfect book to read in one short sentence"),
        HumanMessage(content="I enjoy mystery novels, what should I read?")
    ]
msg = llm.invoke( current_messages)
print(msg)
current_messages.append(msg)
current_messages.append(HumanMessage(content="I also like science fiction, what do you recommend?"))

llm.invoke(current_messages).pretty_print()  

content='You might enjoy "Gone Girl" by Gillian Flynn, a twisty and suspenseful thriller about a marriage that takes a dark and unexpected turn.' additional_kwargs={} response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-07-25T11:25:30.807778298Z', 'done': True, 'done_reason': 'stop', 'total_duration': 395294223, 'load_duration': 22384847, 'prompt_eval_count': 46, 'prompt_eval_duration': 3591563, 'eval_count': 32, 'eval_duration': 368650813, 'model_name': 'llama3.1:latest'} id='run--f9cb64dd-d80a-448c-9182-4c4f6b549fe3-0' usage_metadata={'input_tokens': 46, 'output_tokens': 32, 'total_tokens': 78}
================================== Ai Message ==================================

Consider reading "Dune" by Frank Herbert, a classic sci-fi novel set in a distant future where humans have colonized other planets, exploring themes of politics, ecology, and human nature.


## Prompt templates

In [11]:
from langchain_core.prompts import PromptTemplate

In [12]:
prompt = PromptTemplate.from_template("Tell me one {adjective} joke about {topic}")
inputs = [{"adjective": "funny", "topic": "cats"}, {"adjective": "silly", "topic": "dogs"}, {"adjective": "hilarious", "topic": "birds"}] 

In [30]:
chain = prompt | llm

for input in inputs:
    print(chain.invoke(input).content)
    print("---")

Why did the cat join a band?

Because it wanted to be the purr-cussionist! (get it?)
---
Here's one:

Why did the dog go to the vet?

Because he was feeling ruff!

Hope that made you howl with laughter! Do you want another one?
---
Why did the bird go to the doctor?

Because it had a fowl cough! (get it?)
---


In [24]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
 ("system", "You are a helpful assistant"),
 ("user", "Tell me a joke about {topic}")
])
input_ = {"topic": "cats"}
prompt.invoke(input_)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

In [40]:
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
("system", "You are a helpful assistant"),
MessagesPlaceholder("msgs")  # This will be replaced with one or more messages
])

input_ = {"msgs": [HumanMessage(content="What is the day after Tuesday?"), AIMessage(content="huh?"), HumanMessage(content="Also whats the day after Wednesday?")]}
prompt.invoke(input_)
print(prompt.invoke(input_))
chain = prompt | llm
chain.invoke(input_).pretty_print()

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the day after Tuesday?', additional_kwargs={}, response_metadata={}), AIMessage(content='huh?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Also whats the day after Wednesday?', additional_kwargs={}, response_metadata={})]
================================== Ai Message ==================================

The day after Tuesday is Wednesday.

And, similarly, the day after Wednesday is Thursday.


### Output Parsers

In [51]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

joke_query = "Tell me a joke."

output_parser = JsonOutputParser(pydantic_object=Joke)

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # Dynamic variables that will be provided when invoking the chain
    partial_variables={"format_instructions": format_instructions},  # Static variables set once when creating the prompt
)

chain = prompt | llm | output_parser
chain.invoke({"query": joke_query})



{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

try to create a loop using a pydantic and chain for multiple jokes with various adjectives and animals

In [69]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class KnockKnockJoke(BaseModel):
    topic: str = Field(description="topic of the knock-knock joke")
    response: str = Field(description="the knock-knock joke response")

output_parser1 = JsonOutputParser(pydantic_object=KnockKnockJoke)
output_parser2 = CommaSeparatedListOutputParser(pydantic_object=KnockKnockJoke)

format_instructions1 = output_parser1.get_format_instructions()
format_instructions2 = output_parser2.get_format_instructions()


prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\nTell me a knock knock joke about {topic}\n",
    input_variables=["query"],  # Dynamic variables that will be provided when invoking the chain
    partial_variables={"format_instructions": format_instructions1},  # Static variables set once when creating the prompt
)


topics = ["cats", "dogs", "birds"]

for topic in topics:
    chain = prompt | llm | output_parser1
    print(chain.invoke({"topic": topic}))
    print("---")




{'topic': 'Cats', 'response': "Knock, knock! Who's there? Purr. Purr who? Purrrr-haps you'll let me in?"}
---
{'topic': 'Dogs', 'response': "Knock, knock! Who's there? Arf. Arf who? Dog gone it, I forgot the punchline!"}
---
{'topic': 'Birds', 'response': "Knock, knock! Who's there? Bird. Bird who? Fowl mood today!"}
---


In [70]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\nTell me a knock knock joke about {topic}\n",
    input_variables=["query"],  # Dynamic variables that will be provided when invoking the chain
    partial_variables={"format_instructions": format_instructions2},  # Static variables set once when creating the prompt
)


topics = ["cats", "dogs", "birds"]

for topic in topics:
    chain = prompt | llm | output_parser2
    print(chain.invoke({"topic": topic}))
    print("---")



['Cat', "Purr-haps you'll let me in?"]
---
['Knock knock', "who's there?", 'Ruff', 'Ruff who?', 'Ruff day without treats.']
---
['Birds', 'chirping', 'flying high']
---


### CSV output parser - recipe suggestions

In [75]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate

# Define Pydantic model
class Recipe(BaseModel):
    name: str = Field(description="name of the recipe")
    cuisine: str = Field(description="cuisine type of the recipe")
    ingredients: str = Field(description="list of ingredients for the recipe")

# Parsers
json_parser = JsonOutputParser(pydantic_object=Recipe)
csv_parser = CommaSeparatedListOutputParser()

# Prompts
prompt1 = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\nProvide a recipe belonging to a certain cuisine and the required ingredient for the following country: {country}",
    input_variables=["country"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)

prompt2 = PromptTemplate(
    template="Answer the query.\n{format_instructions_csv}\nGiven the recipe name {recipe}, provide a list of required ingredients. Do not add any introduction or explanation.",
    input_variables=["recipe"],
    partial_variables={"format_instructions_csv": csv_parser.get_format_instructions()},
)

# Extract recipe name from parsed JSON
extract_recipe_name = RunnableLambda(lambda x: {"recipe": Recipe(**x).name} if isinstance(x, dict) else {"recipe": x.name})


# Build chain
full_chain = (
    prompt1
    | llm
    | json_parser
    | extract_recipe_name
    | prompt2
    | llm
    | csv_parser
)

# Run chain
for country in ["India", "Italy", "Mexico"]:
    result = full_chain.invoke({"country": country})
    print(f"{country}: {result}")
    print("---")


India: ['Chicken breast', 'Yogurt', 'Lemon juice', 'Garam masala', 'Cumin powder', 'Coriander powder', 'Cayenne pepper', 'Salt', 'Black pepper', 'Vegetable oil', 'Tomato puree', 'Heavy cream', 'Butter', 'Garlic', 'Ginger', 'Onion', 'Fresh cilantro']
---
Italy: ['Ground beef', 'Onion', 'Garlic', 'Carrot', 'Celery', 'Tomato paste', 'Canned tomatoes', 'Olive oil', 'Salt', 'Black pepper', 'Italian seasoning', 'Red wine (optional)', 'Parmesan cheese', 'Spaghetti']
---
Mexico: ['Pork shoulder', 'Onion', 'Garlic', 'Cumin', 'Coriander', 'Chili powder', 'Oregano', 'Lime juice', 'Olive oil', 'Pineapple chunks', 'Red onion', 'Cilantro', 'Salt', 'Pepper', 'Tortillas', 'Pita bread', 'Skewers']
---


In [76]:
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

# Set up parsers
recipe_parser = StrOutputParser()
csv_parser = CommaSeparatedListOutputParser()

# Prompts
prompt1 = PromptTemplate.from_template("Provide only the name of a famous recipe from {country}. Do not add any other text.")
prompt2 = PromptTemplate.from_template(
    "Answer the query.\n{format_instructions_csv}\nGiven the recipe name {recipe}, provide a list of required ingredients. Do not add any introduction or explanation."
)

# Add format instructions
prompt2 = prompt2.partial(format_instructions_csv=csv_parser.get_format_instructions())

# Chain pieces
get_recipe_name = prompt1 | llm | recipe_parser
store_recipe_name = RunnableLambda(lambda name: {"recipe": name.strip()})

# Create a full pipeline that keeps all data
full_chain = (
    RunnableLambda(lambda x: {"country": x["country"]})  # Step 1 input
    | RunnableLambda(lambda x: {**x, "recipe": get_recipe_name.invoke(x)})  # Add recipe name
    | RunnableLambda(lambda x: {**x, "ingredients": csv_parser.invoke(llm.invoke(prompt2.format(recipe=x["recipe"])))})  # Add ingredients
)

# Run
for country in ["India", "Italy", "Mexico"]:
    result = full_chain.invoke({"country": country})
    print(result)


{'country': 'India', 'recipe': 'Butter Chicken', 'ingredients': ['Tomato puree', 'Yogurt', 'Garam masala', 'Cumin powder', 'Coriander powder', 'Red chili powder', 'Salt', 'Ginger paste', 'Garlic paste', 'Lemon juice', 'Butter', 'Vegetable oil', 'Chicken breast', 'Kasoori methi', 'Fresh cilantro']}
{'country': 'Italy', 'recipe': 'Spaghetti Carbonara', 'ingredients': ['Spaghetti', 'bacon', 'eggs', 'parmesan cheese', 'black pepper', 'salt']}
{'country': 'Mexico', 'recipe': 'Chiles Rellenos', 'ingredients': ['Roasted poblano peppers', 'Queso Oaxaca cheese', 'beaten eggs', 'milk', 'all-purpose flour', 'vegetable oil', 'Salt', 'Pepper', 'Onion', 'Garlic', 'Tomato', 'Fresh cilantro']}
